# Extracting Text from Images in Python

<img src="https://media.arxiv-vanity.com/render-output/6158804/images/fig_2_no_trademarks.png" alt="text to images" width="600" height="600">


In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from PIL import Image

plt.style.use('ggplot')

In [ ]:
#SequentialMatcher sample code: https://www.appsloveworld.com/pandas/100/327/comparing-strings-within-two-columns-in-pandas-with-sequencematcher

# Outline
1. Take a look at the data
2. Extract text from images:
    - pytesseract
    - easyocr
    - keras_ocr
3. Run on a few examples and compare the results

In [ ]:
annot = pd.read_parquet('../input/textocr-text-extraction-from-images-dataset/annot.parquet')
imgs = pd.read_parquet('../input/textocr-text-extraction-from-images-dataset/img.parquet')
img_fns = glob('../input/textocr-text-extraction-from-images-dataset/train_val_images/train_images/*')

# Plot Example Images

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(plt.imread(img_fns[12]))
ax.axis('off')
plt.show()

In [ ]:
image_id = img_fns[0].split('/')[-1].split('.')[0]
annot.query('image_id == @image_id')

# Method 1: pytesseract

In [ ]:
import pytesseract

# Example call
print(pytesseract.image_to_string(img_fns[12], lang='eng'))

In [ ]:
len(img_fns)

In [ ]:
####Loop through each image to extract text and save in df. Meanwhile calculate run time
import pytesseract
from pytesseract import Output
from tqdm import tqdm, trange
from difflib import SequenceMatcher

# record start time 
import timeit
start_time = timeit.default_timer()


#create an empty dataframe
df=pd.DataFrame(columns = ['image_id', 'text'])

#loop through each image and append results
for i in trange(len(img_fns)):
    image_id = img_fns[i].split('/')[-1].rstrip('.jpg')
    data=pytesseract.image_to_data(img_fns[i],output_type=Output.DATAFRAME )
    data['image_id']=image_id
    df=df.append(data[['image_id','text']],ignore_index=True)

# caculate the run time in sec
elapsed = timeit.default_timer() - start_time

#remove NaN and missing values from df
df1=df[df.text!=' '].dropna()

#concatenate text field by groupby image_id
df2=df1.sort_values('text').groupby(['image_id'], as_index=False).agg({'text': ' '.join})

annot_agg=annot[['image_id','utf8_string']].sort_values('utf8_string').groupby(['image_id'], as_index=False).agg({'utf8_string': ' '.join})

final_p=annot_agg.merge(df2, how='left', left_on='image_id',right_on='image_id')

def apply_sm(df):
    if not df[['utf8_string', 'text']].isnull().any():
        return SequenceMatcher(None, df['utf8_string'], df['text']).ratio()
    return 0.0
final_p['ratio']=final_p.apply(apply_sm,axis=1)
final_p["ratio"].mean()

In [ ]:
elapsed/3600

# Method 2: easyocr

In [ ]:
####Loop through each image to extract text and save in df. Meanwhile calculate run time
import easyocr
reader = easyocr.Reader(['en'], gpu = True)

# record start time 
import timeit
start_time = timeit.default_timer()


#create an empty dataframe
df=pd.DataFrame(columns = ['image_id', 'text'])

#loop through each image and append results
for i in trange(len(img_fns)):
    image_id = img_fns[i].split('/')[-1].rstrip('.jpg')
    results = reader.readtext(img_fns[i])
    data=pd.DataFrame(results, columns=['bbox','text','conf'])
    data['image_id']=image_id
    df=df.append(data[['image_id','text']],ignore_index=True)

# caculate the run time in sec
elapsed = timeit.default_timer() - start_time


#remove NaN and missing values from df
df1=df[df.text!=' '].dropna()

#concatenate text field by groupby image_id
df2=df1.sort_values('text').groupby(['image_id'], as_index=False).agg({'text': ' '.join})

annot_agg=annot[['image_id','utf8_string']].sort_values('utf8_string').groupby(['image_id'], as_index=False).agg({'utf8_string': ' '.join})

final_p=annot_agg.merge(df2, how='left', left_on='image_id',right_on='image_id')

def apply_sm(df):
    if not df[['utf8_string', 'text']].isnull().any():
        return SequenceMatcher(None, df['utf8_string'], df['text']).ratio()
    return 0.0
final_p['ratio']=final_p.apply(apply_sm,axis=1)
final_p["ratio"].mean()



# Method 3: keras_ocr

In [ ]:
!pip install keras-ocr -q

In [ ]:
import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

In [ ]:
results = pipeline.recognize([img_fns[11]])

In [ ]:
pd.DataFrame(results[0], columns=['text', 'bbox'])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
keras_ocr.tools.drawAnnotations(plt.imread(img_fns[11]), results[0], ax=ax)
ax.set_title('Keras OCR Result Example')
plt.show()

# Compare easyocr vs. keras_ocr

In [ ]:
# easyocr
reader = easyocr.Reader(['en'], gpu = True)

dfs = []
for img in tqdm(img_fns[:25]):
    result = reader.readtext(img)
    img_id = img.split('/')[-1].split('.')[0]
    img_df = pd.DataFrame(result, columns=['bbox','text','conf'])
    img_df['img_id'] = img_id
    dfs.append(img_df)
easyocr_df = pd.concat(dfs)

In [ ]:
# keras_ocr
pipeline = keras_ocr.pipeline.Pipeline()

dfs = []
for img in tqdm(img_fns[:25]):
    results = pipeline.recognize([img])
    result = results[0]
    img_id = img.split('/')[-1].split('.')[0]
    img_df = pd.DataFrame(result, columns=['text', 'bbox'])
    img_df['img_id'] = img_id
    dfs.append(img_df)
kerasocr_df = pd.concat(dfs)

# Plot Results: easyocr vs keras_ocr

In [ ]:
def plot_compare(img_fn, easyocr_df, kerasocr_df):
    img_id = img_fn.split('/')[-1].split('.')[0]
    fig, axs = plt.subplots(1, 2, figsize=(15, 10))

    easy_results = easyocr_df.query('img_id == @img_id')[['text','bbox']].values.tolist()
    easy_results = [(x[0], np.array(x[1])) for x in easy_results]
    keras_ocr.tools.drawAnnotations(plt.imread(img_fn), 
                                    easy_results, ax=axs[0])
    axs[0].set_title('easyocr results', fontsize=24)

    keras_results = kerasocr_df.query('img_id == @img_id')[['text','bbox']].values.tolist()
    keras_results = [(x[0], np.array(x[1])) for x in keras_results]
    keras_ocr.tools.drawAnnotations(plt.imread(img_fn), 
                                    keras_results, ax=axs[1])
    axs[1].set_title('keras_ocr results', fontsize=24)
    plt.show()

In [ ]:
# Loop over results
for img_fn in img_fns[:25]:
    plot_compare(img_fn, easyocr_df, kerasocr_df)

# The End